In [1]:
# Calcula la temperatura media anual.

import os

import pandas as pd
import numpy as np

import xarray as xr

from scipy import integrate

In [2]:
path_d = "../../../resultados/CHIRTS/"
path_r = "../../../resultados/vars/"

# Variables de CHIRTS.
vars = ["Tmax", "Tmin", "Tmean"]

# Si no existe la carpeta, la crea.
if not os.path.exists(path_r):
    os.mkdir(path_r)

In [6]:
varname = ["CDD_mean", "HDD_mean"]

clim_mean = xr.open_dataset(path_d + "mexico_"
    + vars[2] + ".nc").rename(Tmean = "degree_day")

# Si no existen las carpeta, las crea.
if not os.path.exists(path_r + varname[0]):
    os.mkdir(path_r + varname[0])
if not os.path.exists(path_r + varname[1]):
    os.mkdir(path_r + varname[1])

for j in range(2010, 2017):
    with ( xr.open_dataset( path_d + vars[2] + "/mexico_" 
        + vars[2] + "." + str(j) + ".nc" )
        .rename(Tmean = "degree_day") ) as ds_mean:
        # Establecemos np.nan como valor sin datos.
        ds_mean = ds_mean.where(ds_mean > -9000)
        ds = ds_mean - clim_mean

    ds_CDD = ds.where(ds >= 0)
    ds_CDD.to_netcdf(path_r + varname[0] + "/mexico_" 
        + varname[0] + "." + str(j) + ".nc" )
    ds_HDD = ds.where(ds <= 0)
    ds_HDD.to_netcdf(path_r + varname[1] + "/mexico_" 
        + varname[1] + "." + str(j) + ".nc" )

In [7]:
varname = "CDD_p90"

clim_mean = xr.open_dataset(path_d + "mexico_"
    + vars[0] + ".nc").rename(Tmean = "degree_day")

# Si no existe la carpeta, la crea.
if not os.path.exists(path_r + varname):
    os.mkdir(path_r + varname)

for j in range(2010, 2017):
    with ( xr.open_dataset( path_d + vars[2] + "/mexico_" 
        + vars[2] + "." + str(j) + ".nc" )
        .rename(Tmean = "degree_day") ) as ds_mean:
        # Establecemos np.nan como valor sin datos.
        ds_mean = ds_mean.where(ds_mean > -9000)
        ds = ds_mean - clim_mean

    ds_CDD = ds.where(ds >= 0)
    ds_CDD.to_netcdf(path_r + varname + "/mexico_" 
        + varname + "." + str(j) + ".nc" )

In [ ]:
varname = "HDD_p10"

clim_mean = xr.open_dataset(path_d + "mexico_"
    + vars[1] + ".nc").rename(Tmean = "degree_day")

# Si no existe la carpeta, la crea.
if not os.path.exists(path_r + varname):
    os.mkdir(path_r + varname)

for j in range(2010, 2017):
    with ( xr.open_dataset( path_d + vars[2] + "/mexico_" 
        + vars[2] + "." + str(j) + ".nc" )
        .rename(Tmean = "degree_day") ) as ds_mean:
        # Establecemos np.nan como valor sin datos.
        ds_mean = ds_mean.where(ds_mean > -9000)
        ds = ds_mean - clim_mean

    ds_HDD = ds.where(ds <= 0)
    ds_HDD.to_netcdf(path_r + varname + "/mexico_" 
        + varname + "." + str(j) + ".nc" )